It was reported by the MARS team that py-earth is outdated. The MARS team resorted to suing spline then linear regression pipeline. I will adopt this decision, I might also look into if I could use rpy2 to use mars from r instead (if I have time). 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [1]:
import sys 
sys.path.append("../../")
from proj_mod import drop_extreme
from proj_mod import data_shifting

In [4]:
all_data=pd.read_csv("../../data/processed/all_data_collection.csv")

In [5]:
all_data_extended = all_data.copy(deep=True)

In [6]:
lst_cols_shift=['tech_index_daily_pro_change', 'cop_daily_pro_change',
       'gold_daily_pro_change', 'silv_daily_pro_change',
       'pal_daily_pro_change', 'plat_daily_pro_change',
       'crude_oil_daily_pro_change']

To my knowledge, people normally use degree 3 with spline polynomial degree. In experience with EBM, it seems that 10 shifting provides a better "slope" to the prediction. 

In [8]:
from sklearn.preprocessing import SplineTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [7]:
all_data_extended = all_data.copy(deep=True)
for int_extend in range(11): 
    if int_extend != 0: 
        for col in lst_cols_shift: 
            all_data_extended=data_shifting.shifter(df_in=all_data_extended,str_col=col,int_shift=int_extend)
all_data_extended_train=all_data_extended[all_data_extended["Date"]<"2024-01-01"].dropna()
all_data_extended_train_de=drop_extreme.drop_by_sort(df_in=all_data_extended_train,fl_low=0.001,fl_high=0.999)
all_data_extended_test=all_data_extended[all_data_extended["Date"]>="2024-01-01"].dropna()
all_data_extended_test_de=drop_extreme.drop_by_sort(df_in=all_data_extended_test,fl_low=0.01,fl_high=0.99)
X_train=all_data_extended_train.iloc[:,2:]
y_train=all_data_extended_train.iloc[:,1]
X_de_train=all_data_extended_train_de.iloc[:,2:]
y_de_train=all_data_extended_train_de.iloc[:,1]
X_test=all_data_extended_test.iloc[:,2:]
y_test=all_data_extended_test.iloc[:,1]
X_de_test=all_data_extended_test_de.iloc[:,2:]
y_de_test=all_data_extended_test_de.iloc[:,1]

In [9]:
def create_sp_pipe(int_knots,int_deg): 
    return Pipeline([
    ("std", StandardScaler()),  
    ("spline", SplineTransformer(n_knots=int_knots, degree=int_deg)),
    ("regressor", LinearRegression())          
])


At first, I would like to do cross validation on knots only, or degree only, but it seems that, after some testing, only knots=2 and deg=1 gives the best outcome, this is almost the same as simply running a linear regression. 

In [16]:
arr_mse_kfold_collection=[] #Each row is for one int_extend. Then for each row index zero is for the training with extreme values, and index one is for the training without extreme values. 
arr_mse_kfold_de_collection=[] #Each row is for one int_extend. Then for each row index zero is for the training with extreme values, and index one is for the training without extreme values. 
for int_i in range(2,6): 
    # if int_extend != 0: 
    #     for col in lst_cols_shift: 
    #         all_data_extended=data_shifting.shifter(df_in=all_data_extended,str_col=col,int_shift=int_extend)
    all_data_extended_train=all_data_extended[all_data_extended["Date"]<"2024-01-01"].dropna()
    # all_data_extended_de=drop_extreme.drop_by_sort(df_in=all_data_extended.dropna(),fl_low=0.001,fl_high=0.999)
    # all_data_extended_train_de=drop_extreme.drop_by_sort(df_in=all_data_extended_train,fl_low=0.001,fl_high=0.999)
    # all_data_extended_test=all_data_extended[all_data_extended["Date"]>="2024-01-01"].dropna()
    X_train=all_data_extended_train.iloc[:,2:]
    y_train=all_data_extended_train.iloc[:,1]
    # X_de_train=all_data_extended_train_de.iloc[:,2:]
    # y_de_train=all_data_extended_train_de.iloc[:,1]
    # X_test=all_data_extended_test.iloc[:,2:]
    # y_test=all_data_extended_test.iloc[:,1]
    # ebm_pipe=Pipeline([("std",StandardScaler()),("ebm",ExplainableBoostingRegressor())])
    time_kfold=TimeSeriesSplit(n_splits=5)
    fold_mse=[]
    fold_mse_de=[]
    #First for without deleting extreme values. 
    for index_train, index_test in time_kfold.split(X_train): 
        X_tt=X_train.iloc[index_train]
        y_tt=y_train.iloc[index_train]
        X_ho=X_train.iloc[index_test]
        y_ho=y_train.iloc[index_test]
        all_data_ho_de=drop_extreme.drop_by_sort(df_in=all_data_extended_train.iloc[index_test], fl_low=0.01, fl_high=0.99)
        X_ho_de=all_data_ho_de.iloc[:,2:]
        y_ho_de=all_data_ho_de.iloc[:,1]
        sp_pipe=create_sp_pipe(int_knots=int_i,int_deg=1)
        sp_pipe.fit(X=X_tt,y=y_tt)
        # ebm_pipe.fit(X=X_tt,y=y_tt)
        pred=sp_pipe.predict(X=X_ho)
        # pred=ebm_pipe.predict(X=X_ho)
        error=mean_squared_error(y_pred=pred,y_true=y_ho)
        fold_mse.append(error)
        pred_de=sp_pipe.predict(X=X_ho_de)
        # pred_de=ebm_pipe.predict(X=X_ho_de)
        error_de=mean_squared_error(y_pred=pred_de,y_true=y_ho_de)
        fold_mse_de.append(error_de)
    mean_error=np.mean(fold_mse)
    arr_mse_kfold_collection.append([])
    arr_mse_kfold_collection[-1].append(mean_error)
    mean_error_de=np.mean(fold_mse_de)
    arr_mse_kfold_de_collection.append([])
    arr_mse_kfold_de_collection[-1].append(mean_error_de)
    print("mse trained on "+str(int_i)+" knots and WITHOUT deleting extreme values is "+str(mean_error)+" with k fold.")
    fold_mse=[]
    fold_mse_de=[]
    #Then the one with deleting extreme values. 
    for index_train, index_test in time_kfold.split(X_train): 
        all_data_tt=drop_extreme.drop_by_sort(df_in=all_data_extended_train.iloc[index_train], fl_low=0.001, fl_high=0.999)  #all_data_extended_train.iloc[index_train]
        all_data_ho=drop_extreme.drop_by_sort(df_in=all_data_extended_train.iloc[index_test], fl_low=0.01, fl_high=0.99) #all_data_extended_train.iloc[index_test]
        X_tt=all_data_tt.iloc[:,2:] #drop_extreme.drop_by_sort(df_in=X_train.iloc[index_train], fl_low=0.01, fl_high=0.99)     #X_de_train.iloc[index_train]
        y_tt=all_data_tt.iloc[:,1] #drop_extreme.drop_by_sort(df_in=y_train.iloc[index_train], fl_low=0.01, fl_high=0.99) 
        X_ho_de=all_data_ho.iloc[:,2:] #drop_extreme.drop_by_sort(df_in=X_train.iloc[index_test], fl_low=0.01, fl_high=0.99) #X_train.iloc[index_test]
        y_ho_de=all_data_ho.iloc[:,1] #drop_extreme.drop_by_sort(df_in=y_train.iloc[index_test], fl_low=0.01, fl_high=0.99) #y_train.iloc[index_test]
        X_ho=X_train.iloc[index_test]
        y_ho=y_train.iloc[index_test]
        sp_pipe=create_sp_pipe(int_knots=int_i,int_deg=1)
        sp_pipe.fit(X=X_tt,y=y_tt)
        # ebm_pipe.fit(X=X_tt,y=y_tt)
        pred=sp_pipe.predict(X=X_ho)
        # pred=ebm_pipe.predict(X=X_ho)
        error=mean_squared_error(y_pred=pred,y_true=y_ho)
        fold_mse.append(error)
        pred_de=sp_pipe.predict(X=X_ho_de)
        # pred_de=ebm_pipe.predict(X=X_ho_de)
        error_de=mean_squared_error(y_pred=pred_de,y_true=y_ho_de)
        fold_mse_de.append(error_de)
    mean_error=np.mean(fold_mse)
    arr_mse_kfold_collection[-1].append(mean_error)
    mean_error_de=np.mean(fold_mse_de)
    arr_mse_kfold_de_collection[-1].append(mean_error_de)
    print("mse trained on "+str(int_i)+" knots and WITH deleting extreme values is "+str(mean_error)+" with k fold.")
    print("\n"+"Finished training and testing for "+str(int_i)+" knots.")

mse trained on 2 knots and WITHOUT deleting extreme values is 2.242200672138371 with k fold.
mse trained on 2 knots and WITH deleting extreme values is 2.27261045873911 with k fold.

Finished training and testing for 2 knots.
mse trained on 3 knots and WITHOUT deleting extreme values is 3.271700838033316 with k fold.
mse trained on 3 knots and WITH deleting extreme values is 3.3637519156386455 with k fold.

Finished training and testing for 3 knots.
mse trained on 4 knots and WITHOUT deleting extreme values is 6.341562078653881 with k fold.
mse trained on 4 knots and WITH deleting extreme values is 5.644421991651738 with k fold.

Finished training and testing for 4 knots.
mse trained on 5 knots and WITHOUT deleting extreme values is 130.06541873993166 with k fold.
mse trained on 5 knots and WITH deleting extreme values is 128.40649659371905 with k fold.

Finished training and testing for 5 knots.


In [18]:
arr_mse_kfold_collection=[] #Each row is for one int_extend. Then for each row index zero is for the training with extreme values, and index one is for the training without extreme values. 
arr_mse_kfold_de_collection=[] #Each row is for one int_extend. Then for each row index zero is for the training with extreme values, and index one is for the training without extreme values. 
for int_i in range(1,6): 
    # if int_extend != 0: 
    #     for col in lst_cols_shift: 
    #         all_data_extended=data_shifting.shifter(df_in=all_data_extended,str_col=col,int_shift=int_extend)
    all_data_extended_train=all_data_extended[all_data_extended["Date"]<"2024-01-01"].dropna()
    # all_data_extended_de=drop_extreme.drop_by_sort(df_in=all_data_extended.dropna(),fl_low=0.001,fl_high=0.999)
    # all_data_extended_train_de=drop_extreme.drop_by_sort(df_in=all_data_extended_train,fl_low=0.001,fl_high=0.999)
    # all_data_extended_test=all_data_extended[all_data_extended["Date"]>="2024-01-01"].dropna()
    X_train=all_data_extended_train.iloc[:,2:]
    y_train=all_data_extended_train.iloc[:,1]
    # X_de_train=all_data_extended_train_de.iloc[:,2:]
    # y_de_train=all_data_extended_train_de.iloc[:,1]
    # X_test=all_data_extended_test.iloc[:,2:]
    # y_test=all_data_extended_test.iloc[:,1]
    # ebm_pipe=Pipeline([("std",StandardScaler()),("ebm",ExplainableBoostingRegressor())])
    time_kfold=TimeSeriesSplit(n_splits=5)
    fold_mse=[]
    fold_mse_de=[]
    #First for without deleting extreme values. 
    for index_train, index_test in time_kfold.split(X_train): 
        X_tt=X_train.iloc[index_train]
        y_tt=y_train.iloc[index_train]
        X_ho=X_train.iloc[index_test]
        y_ho=y_train.iloc[index_test]
        all_data_ho_de=drop_extreme.drop_by_sort(df_in=all_data_extended_train.iloc[index_test], fl_low=0.01, fl_high=0.99)
        X_ho_de=all_data_ho_de.iloc[:,2:]
        y_ho_de=all_data_ho_de.iloc[:,1]
        sp_pipe=create_sp_pipe(int_knots=2,int_deg=int_i)
        sp_pipe.fit(X=X_tt,y=y_tt)
        # ebm_pipe.fit(X=X_tt,y=y_tt)
        pred=sp_pipe.predict(X=X_ho)
        # pred=ebm_pipe.predict(X=X_ho)
        error=mean_squared_error(y_pred=pred,y_true=y_ho)
        fold_mse.append(error)
        pred_de=sp_pipe.predict(X=X_ho_de)
        # pred_de=ebm_pipe.predict(X=X_ho_de)
        error_de=mean_squared_error(y_pred=pred_de,y_true=y_ho_de)
        fold_mse_de.append(error_de)
    mean_error=np.mean(fold_mse)
    arr_mse_kfold_collection.append([])
    arr_mse_kfold_collection[-1].append(mean_error)
    mean_error_de=np.mean(fold_mse_de)
    arr_mse_kfold_de_collection.append([])
    arr_mse_kfold_de_collection[-1].append(mean_error_de)
    print("mse trained on "+str(int_i)+" degree and WITHOUT deleting extreme values is "+str(mean_error)+" with k fold.")
    fold_mse=[]
    fold_mse_de=[]
    #Then the one with deleting extreme values. 
    for index_train, index_test in time_kfold.split(X_train): 
        all_data_tt=drop_extreme.drop_by_sort(df_in=all_data_extended_train.iloc[index_train], fl_low=0.001, fl_high=0.999)  #all_data_extended_train.iloc[index_train]
        all_data_ho=drop_extreme.drop_by_sort(df_in=all_data_extended_train.iloc[index_test], fl_low=0.01, fl_high=0.99) #all_data_extended_train.iloc[index_test]
        X_tt=all_data_tt.iloc[:,2:] #drop_extreme.drop_by_sort(df_in=X_train.iloc[index_train], fl_low=0.01, fl_high=0.99)     #X_de_train.iloc[index_train]
        y_tt=all_data_tt.iloc[:,1] #drop_extreme.drop_by_sort(df_in=y_train.iloc[index_train], fl_low=0.01, fl_high=0.99) 
        X_ho_de=all_data_ho.iloc[:,2:] #drop_extreme.drop_by_sort(df_in=X_train.iloc[index_test], fl_low=0.01, fl_high=0.99) #X_train.iloc[index_test]
        y_ho_de=all_data_ho.iloc[:,1] #drop_extreme.drop_by_sort(df_in=y_train.iloc[index_test], fl_low=0.01, fl_high=0.99) #y_train.iloc[index_test]
        X_ho=X_train.iloc[index_test]
        y_ho=y_train.iloc[index_test]
        sp_pipe=create_sp_pipe(int_knots=2,int_deg=int_i)
        sp_pipe.fit(X=X_tt,y=y_tt)
        # ebm_pipe.fit(X=X_tt,y=y_tt)
        pred=sp_pipe.predict(X=X_ho)
        # pred=ebm_pipe.predict(X=X_ho)
        error=mean_squared_error(y_pred=pred,y_true=y_ho)
        fold_mse.append(error)
        pred_de=sp_pipe.predict(X=X_ho_de)
        # pred_de=ebm_pipe.predict(X=X_ho_de)
        error_de=mean_squared_error(y_pred=pred_de,y_true=y_ho_de)
        fold_mse_de.append(error_de)
    mean_error=np.mean(fold_mse)
    arr_mse_kfold_collection[-1].append(mean_error)
    mean_error_de=np.mean(fold_mse_de)
    arr_mse_kfold_de_collection[-1].append(mean_error_de)
    print("mse trained on "+str(int_i)+" degree and WITH deleting extreme values is "+str(mean_error)+" with k fold.")
    print("\n"+"Finished training and testing for "+str(int_i)+" degree.")

mse trained on 1 degree and WITHOUT deleting extreme values is 2.242200672138371 with k fold.
mse trained on 1 degree and WITH deleting extreme values is 2.27261045873911 with k fold.

Finished training and testing for 1 degree.
mse trained on 2 degree and WITHOUT deleting extreme values is 3.4615019536372067 with k fold.
mse trained on 2 degree and WITH deleting extreme values is 3.4985422253026046 with k fold.

Finished training and testing for 2 degree.
mse trained on 3 degree and WITHOUT deleting extreme values is 8.137809352118662 with k fold.
mse trained on 3 degree and WITH deleting extreme values is 8.539543410164157 with k fold.

Finished training and testing for 3 degree.
mse trained on 4 degree and WITHOUT deleting extreme values is 180.0526143199764 with k fold.
mse trained on 4 degree and WITH deleting extreme values is 180.06825253352437 with k fold.

Finished training and testing for 4 degree.
mse trained on 5 degree and WITHOUT deleting extreme values is 56900.928428178

This should be the reason that why the MARS team only looked at knots=2, degree=1. 

If I want anything more interesting, I should go look at how to use mars with r. 